In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Carregando os dados
df_breastcancer = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AM/lista-2/breastcancer.csv")

# Dividindo os dados em características (X) e rótulo (y)
X = df_breastcancer.iloc[:, :-1].values  # 30 primeiras colunas são características
y = df_breastcancer.iloc[:, -1].values.reshape(-1, 1)  # Última coluna é a saída

# Normalizando os dados de entrada
X_normalized = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))


In [ ]:
df_breastcancer

,1.798999999999999844e+01,1.038000000000000078e+01,1.227999999999999972e+02,1.001000000000000000e+03,1.184000000000000052e-01,2.776000000000000134e-01,3.000999999999999779e-01,1.471000000000000085e-01,2.419000000000000039e-01,7.871000000000000218e-02,...,1.732999999999999829e+01,1.845999999999999943e+02,2.019000000000000000e+03,1.622000000000000108e-01,6.655999999999999694e-01,7.118999999999999773e-01,2.654000000000000248e-01,4.601000000000000090e-01,1.189000000000000057e-01,1.000000000000000000e+00
0,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,1.0
1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,1.0
2,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,1.0
3,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,1.0
4,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.17910,0.52490,0.5355,0.1741,0.3985,0.12440,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
563,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,1.0
564,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,1.0
565,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,1.0
566,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,1.0


In [ ]:
# Calculando média e desvio padrão
def mean_std(values):
    return np.mean(values), np.std(values)

In [ ]:
# Função de ativação sigmoidal
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
# Calculando as previsões
def predict(X, weights, bias):
    z = np.dot(X, weights) + bias
    return sigmoid(z)

In [ ]:
# Implementando o modelo gd
def gradient_descent(X, y, weights, bias, learning_rate, iterations):
    n = float(len(X))

    for i in range(iterations):
        y_pred = predict(X, weights, bias)
        error = y - y_pred
        gradient_weights = -np.dot(X.T, error) / n
        gradient_bias = -np.mean(error)
        weights -= learning_rate * gradient_weights
        bias -= learning_rate * gradient_bias

    return weights, bias

In [ ]:
def stochastic_gradient_descent(X, y, weights, bias, learning_rate):
    n = float(len(X))
    for i in range(len(X)):
        x_i = X[i]
        y_i = y[i]
        y_pred = predict(x_i, weights, bias)
        error = y_i - y_pred
        gradient_weights = -x_i.reshape(-1,1) * error
        gradient_bias = -error
        weights -= learning_rate * gradient_weights
        bias -= learning_rate * gradient_bias
    return weights, bias


In [ ]:
# Função para Avaliação do Modelo
def evaluate_model(X_train, y_train, X_test, y_test, learning_rate, iterations, optimizer):
    # Inicializando os pesos e o viés
    weights = np.zeros(X_train.shape[1]).reshape(-1,1)
    bias = 0.0

    # Selecionando o algoritmo de otimização
    if optimizer == 'gd':
        weights, bias = gradient_descent(X_train, y_train, weights, bias, learning_rate, iterations)
    elif optimizer == 'sgd':
        weights, bias = stochastic_gradient_descent(X_train, y_train, weights, bias, learning_rate)

    # Fazendo previsões no conjunto de teste
    y_pred = predict(X_test, weights, bias)
    y_pred_binary = np.round(y_pred)

    # Calculando acurácia global
    accuracy = np.mean(y_pred_binary == y_test)

    # Calculando a acurácia por classe
    class_accuracy_0 = np.mean((y_pred_binary == y_test)[y_test == 0])
    class_accuracy_1 = np.mean((y_pred_binary == y_test)[y_test == 1])

    return accuracy, class_accuracy_0, class_accuracy_1

In [ ]:
# Definindo Hiperparâmetros
learning_rate = 0.1
iterations = 100
n_folds = 10
fold_size = len(X) // n_folds

# Avaliação do Modelo usando Gradient Descent
global_accuracy_list_gd = []
class_accuracy_0_list_gd = []
class_accuracy_1_list_gd = []

# Avaliação do Modelo usando Stochastic Gradient Descent
global_accuracy_list_sgd = []
class_accuracy_0_list_sgd = []
class_accuracy_1_list_sgd = []

for i in range(n_folds):
    start = i * fold_size
    end = (i + 1) * fold_size
    X_test = X_normalized[start:end]
    y_test = y[start:end]
    X_train = np.concatenate([X_normalized[:start], X_normalized[end:]])
    y_train = np.concatenate([y[:start], y[end:]])

    # Avaliando o modelo com Gradient Descent
    global_accuracy_gd, class_accuracy_0_gd, class_accuracy_1_gd = evaluate_model(X_train, y_train, X_test, y_test, learning_rate, iterations, optimizer='gd')

    # Armazenando as métricas de avaliação do GD
    global_accuracy_list_gd.append(global_accuracy_gd)
    class_accuracy_0_list_gd.append(class_accuracy_0_gd)
    class_accuracy_1_list_gd.append(class_accuracy_1_gd)

    # Avaliando o modelo com Stochastic Gradient Descent
    global_accuracy_sgd, class_accuracy_0_sgd, class_accuracy_1_sgd = evaluate_model(X_train, y_train, X_test, y_test, learning_rate, iterations, optimizer='sgd')

    # Armazenando as métricas de avaliação do SGD
    global_accuracy_list_sgd.append(global_accuracy_sgd)
    class_accuracy_0_list_sgd.append(class_accuracy_0_sgd)
    class_accuracy_1_list_sgd.append(class_accuracy_1_sgd)



## Gradient Descent

In [ ]:
# Calculando as métricas de Gradient Descent
mean_global_accuracy_gd, std_global_accuracy_gd = np.mean(global_accuracy_list_gd), np.std(global_accuracy_list_gd)
mean_class_accuracy_0_gd, std_class_accuracy_0_gd = np.mean(class_accuracy_0_list_gd), np.std(class_accuracy_0_list_gd)
mean_class_accuracy_1_gd, std_class_accuracy_1_gd = np.mean(class_accuracy_1_list_gd), np.std(class_accuracy_1_list_gd)

# Imprimindo os resultados do Gradient Descent
print("Gradient Descent:")
print("Média da acurácia global:", mean_global_accuracy_gd)
print("Desvio padrão da acurácia global:", std_global_accuracy_gd)
print("Média da acurácia para classe 0:", mean_class_accuracy_0_gd)
print("Desvio padrão da acurácia para classe 0:", std_class_accuracy_0_gd)
print("Média da acurácia para classe 1:", mean_class_accuracy_1_gd)
print("Desvio padrão da acurácia para classe 1:", std_class_accuracy_1_gd)


Gradient Descent:
Média da acurácia global: 0.8749999999999998
Desvio padrão da acurácia global: 0.13623022230543322
Média da acurácia para classe 0: 0.9947332421340629
Desvio padrão da acurácia para classe 0: 0.010623075833153752
Média da acurácia para classe 1: 0.7360080768904298
Desvio padrão da acurácia para classe 1: 0.15954674052051845


## Stochastic Gradient Descent:

In [ ]:
# Calculando as métricas de Stochastic Gradient Descent
mean_global_accuracy_sgd, std_global_accuracy_sgd = np.mean(global_accuracy_list_sgd), np.std(global_accuracy_list_sgd)
mean_class_accuracy_0_sgd, std_class_accuracy_0_sgd = np.mean(class_accuracy_0_list_sgd), np.std(class_accuracy_0_list_sgd)
mean_class_accuracy_1_sgd, std_class_accuracy_1_sgd = np.mean(class_accuracy_1_list_sgd), np.std(class_accuracy_1_list_sgd)

# Imprimindo os resultados do Stochastic Gradient Descent
print("Stochastic Gradient Descent:")
print("Média da acurácia global:", mean_global_accuracy_sgd)
print("Desvio padrão da acurácia global:", std_global_accuracy_sgd)
print("Média da acurácia para classe 0:", mean_class_accuracy_0_sgd)
print("Desvio padrão da acurácia para classe 0:", std_class_accuracy_0_sgd)
print("Média da acurácia para classe 1:", mean_class_accuracy_1_sgd)
print("Desvio padrão da acurácia para classe 1:", std_class_accuracy_1_sgd)

Stochastic Gradient Descent:
Média da acurácia global: 0.8803571428571428
Desvio padrão da acurácia global: 0.09484479803492502
Média da acurácia para classe 0: 0.9970588235294118
Desvio padrão da acurácia para classe 0: 0.008823529411764707
Média da acurácia para classe 1: 0.7170082858318152
Desvio padrão da acurácia para classe 1: 0.12514489615864807


## Análise do discriminante Gaussiano

In [ ]:
def gaussian_discriminant_analysis(X_train, y_train):
    # Separar os dados por classe
    X_pos = X_train[y_train[:, 0] == 1]
    X_neg = X_train[y_train[:, 0] == 0]

    # Calcular as médias para cada classe
    mu_pos = np.mean(X_pos, axis=0)
    mu_neg = np.mean(X_neg, axis=0)

    # Calcular as matrizes de covariância para cada classe
    cov_pos = np.cov(X_pos, rowvar=False)
    cov_neg = np.cov(X_neg, rowvar=False)

    # Calcular a matriz de covariância comum
    cov_shared = (len(X_pos) / len(X_train)) * cov_pos + (len(X_neg) / len(X_train)) * cov_neg

    # Calcular os termos constantes para a função discriminante
    inv_cov_shared = np.linalg.inv(cov_shared)
    det_cov_shared = np.linalg.det(cov_shared)
    const_pos = -0.5 * np.log(det_cov_shared) + np.log(len(X_pos) / len(X_train))
    const_neg = -0.5 * np.log(det_cov_shared) + np.log(len(X_neg) / len(X_train))

    return mu_pos, mu_neg, inv_cov_shared, const_pos, const_neg


In [ ]:
def predict_gda(X, mu_pos, mu_neg, inv_cov_shared, const_pos, const_neg):
    # Calcular as funções discriminantes para cada classe
    g_pos = np.sum((X - mu_pos) @ inv_cov_shared * (X - mu_pos), axis=1) + const_pos
    g_neg = np.sum((X - mu_neg) @ inv_cov_shared * (X - mu_neg), axis=1) + const_neg

    # Comparar as funções discriminantes e fazer as previsões
    y_pred = (g_pos < g_neg).astype(int)

    return y_pred

In [ ]:
def evaluate_model_gda(X_train, y_train, X_test, y_test):
    # Treinando o modelo GDA
    mu_pos, mu_neg, inv_cov_shared, const_pos, const_neg = gaussian_discriminant_analysis(X_train, y_train)

    # Fazendo previsões no conjunto de teste
    y_pred = predict_gda(X_test, mu_pos, mu_neg, inv_cov_shared, const_pos, const_neg)

    # Calculando acurácia global
    accuracy = np.mean(y_pred == y_test)

    # Calculando a acurácia por classe
    class_accuracy_0 = np.mean((y_pred == y_test)[y_test[:, 0] == 0])
    class_accuracy_1 = np.mean((y_pred == y_test)[y_test[:, 0] == 1])

    return accuracy, class_accuracy_0, class_accuracy_1


In [ ]:
# Inicializando as listas para armazenar as métricas de avaliação
global_accuracy_list_gda = []
class_accuracy_0_list_gda = []
class_accuracy_1_list_gda = []


# Executando a validação cruzada
for i in range(n_folds):
    start = i * fold_size
    end = (i + 1) * fold_size
    X_test = X_normalized[start:end]
    y_test = y[start:end]
    X_train = np.concatenate([X_normalized[:start], X_normalized[end:]])
    y_train = np.concatenate([y[:start], y[end:]])

    # Avaliando o modelo
    global_accuracy, class_accuracy_0, class_accuracy_1 = evaluate_model_gda(X_train, y_train, X_test, y_test)

    # Armazenando as métricas de avaliação
    global_accuracy_list_gda.append(global_accuracy)
    class_accuracy_0_list_gda.append(class_accuracy_0)
    class_accuracy_1_list_gda.append(class_accuracy_1)

# Calculando a média e o desvio padrão das métricas de avaliação
mean_global_accuracy_gda, std_global_accuracy_gda = mean_std(global_accuracy_list_gda)
mean_class_accuracy_0_gda, std_class_accuracy_0_gda = mean_std(class_accuracy_0_list_gda)
mean_class_accuracy_1_gda, std_class_accuracy_1_gda = mean_std(class_accuracy_1_list_gda)

# Imprimindo os resultados
print("Análise do discriminante Gaussiano:")
print("Média da acurácia global:", mean_global_accuracy_gda)
print("Desvio padrão da acurácia global:", std_global_accuracy_gda)
print("Média da acurácia para classe 0:", mean_class_accuracy_0_gda)
print("Desvio padrão da acurácia para classe 0:", std_class_accuracy_0_gda)
print("Média da acurácia para classe 1:", mean_class_accuracy_1_gda)
print("Desvio padrão da acurácia para classe 1:", std_class_accuracy_1_gda)

Análise do discriminante Gaussiano:
Média da acurácia global: 0.5995535714285715
Desvio padrão da acurácia global: 0.07835591654555965
Média da acurácia para classe 0: 0.6589285714285714
Desvio padrão da acurácia para classe 0: 0.15840604309746495
Média da acurácia para classe 1: 0.3410714285714286
Desvio padrão da acurácia para classe 1: 0.15840604309746492


## Naive Bayes

In [ ]:
def fit_naive_bayes(X_train, y_train):
    classes = np.unique(y_train)
    class_priors = {}
    means = {}
    stds = {}
    for c in classes:
        X_c = X_train[y_train.ravel() == c]
        class_priors[c] = len(X_c) / len(X_train)
        means[c] = np.mean(X_c, axis=0)
        stds[c] = np.std(X_c, axis=0)

    return class_priors, means, stds


In [ ]:
def calculate_probability(x, mean, std):
    exponent = np.exp(-((x - mean) ** 2) / (2 * (std ** 2)))
    return np.prod((1 / (np.sqrt(2 * np.pi) * std)) * exponent)

In [ ]:
def predict_naive_bayes(X_test, class_priors, means, stds):
    predictions = []
    for x in X_test:
        class_probabilities = {}
        for c in class_priors:
            class_probabilities[c] = class_priors[c] * calculate_probability(x, means[c], stds[c])
        predicted_class = max(class_probabilities, key=class_probabilities.get)
        predictions.append(predicted_class)

    return np.array(predictions)

In [ ]:
def evaluate_model_naive_bayes(X_train, y_train, X_test, y_test):
    # Treinando o modelo Naive Bayes
    class_priors, means, stds = fit_naive_bayes(X_train, y_train)

    # Fazendo previsões no conjunto de teste
    y_pred = predict_naive_bayes(X_test, class_priors, means, stds)

    # Calculando acurácia global
    accuracy = np.mean(y_pred == y_test)

    # Calculando a acurácia por classe
    class_accuracy_0 = np.mean((y_pred == y_test)[y_test == 0])
    class_accuracy_1 = np.mean((y_pred == y_test)[y_test == 1])

    return accuracy, class_accuracy_0, class_accuracy_1


In [ ]:
# Treinando o modelo Naive Bayes
class_priors, means, stds = fit_naive_bayes(X_normalized, y)

# Realizando a validação cruzada
fold_size = len(X_normalized) // n_folds


# Inicializando as listas para armazenar as métricas de avaliação
global_accuracy_list_nb = []
class_accuracy_0_list_nb = []
class_accuracy_1_list_nb = []

for i in range(n_folds):
    start = i * fold_size
    end = (i + 1) * fold_size
    X_test = X_normalized[start:end]
    y_test = y[start:end].ravel()
    X_train = np.concatenate([X_normalized[:start], X_normalized[end:]])
    y_train = np.concatenate([y[:start], y[end:]])

    accuracy_nb, class_accuracy_0_nb, class_accuracy_1_nb = evaluate_model_naive_bayes(X_train, y_train, X_test, y_test)

    global_accuracy_list_nb.append(accuracy_nb)
    class_accuracy_0_list_nb.append(class_accuracy_0_nb)
    class_accuracy_1_list_nb.append(class_accuracy_1_nb)

# Calculando a média e o desvio padrão das métricas de avaliação
mean_global_accuracy_nb, std_global_accuracy_nb = mean_std(global_accuracy_list_nb)
mean_class_accuracy_0_nb, std_class_accuracy_0_nb = mean_std(class_accuracy_0_list_nb)
mean_class_accuracy_1_nb, std_class_accuracy_1_nb = mean_std(class_accuracy_1_list_nb)


# Imprimindo os resultados
print("Naive Bayes:")
print("Média da acurácia global:", mean_global_accuracy_nb)
print("Desvio padrão da acurácia global:", std_global_accuracy_nb)
print("Média da acurácia para classe 0:", mean_class_accuracy_0_nb)
print("Desvio padrão da acurácia para classe 0:", std_class_accuracy_0_nb)
print("Média da acurácia para classe 1:", mean_class_accuracy_1_nb)
print("Desvio padrão da acurácia para classe 1:", std_class_accuracy_1_nb)

Naive Bayes:
Média da acurácia global: 0.9285714285714286
Desvio padrão da acurácia global: 0.03659625273556998
Média da acurácia para classe 0: 0.9559547056643719
Desvio padrão da acurácia para classe 0: 0.039932124488417604
Média da acurácia para classe 1: 0.8898885428297195
Desvio padrão da acurácia para classe 1: 0.06699303839700968
